In [1]:
import os
from text2img import text_to_image
import concurrent.futures  
def process_item(item, data_dir):  
    question_id = item["question_id"]  
    answers = item['choices'][0]['turns'][0]['content']
    model_id = item["model_id"]
    output_img = f"image_{question_id}.png"  
    data_dir = "data/images"
    text_to_image(answers, output_img, save_dir=model_id, data_dir=data_dir, temp_dir="original_temp")  
    # output_img = f"output_{index}.png"  
    # text_to_image(item["gpt_answer"], output_img, save_dir="gpt_response", temp_dir="gpt_temp")  
    # output_img = f"output_{index}.png"  
    # text_to_image(item["revised_text"], output_img, save_dir="revised_response", temp_dir="revised_temp")  

def generate_image(model_name, model_answers, data_dir):
    max_workers = 20  
    print(f"-----------------generate {model_name} images -----------------")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:  
        futures = [executor.submit(process_item, model_answers[model_name][key], data_dir) for key in model_answers[model_name].keys()]  
        for future in concurrent.futures.as_completed(futures):  
            try:  
                future.result()  
            except Exception as exc:  
                print(f'Generated an exception: {exc}')  
def generate_answer_images(model_answers, model_name, baseline_model, data_dir='data/images'):
    if os.path.exists(os.path.join(data_dir, baseline_model)) == False:
        generate_image(baseline_model, model_answers, data_dir)
    
    #generate images
    generate_image(model_name, model_answers, data_dir)
    
    print("images generated done.")
# model_name = 'Meta-Llama-3.1-8B-Instruct'
# baseline_model = 'gpt4_1106_preview'
# generate_answer_images()


In [2]:
from utils import load_model_answers
answer_dir = 'data/alpaca/model_answer'
model_answers = load_model_answers(answer_dir)
model_name = 'Meta-Llama-3.1-8B-Instruct'
baseline_model = 'gpt4_1106_preview'
generate_answer_images(model_answers, model_name, baseline_model)

2024-09-06 09:27:26,292 - INFO - Starting the conversion process.
2024-09-06 09:27:26,292 - INFO - Starting the conversion process.
2024-09-06 09:27:26,293 - INFO - Starting the conversion process.
2024-09-06 09:27:26,295 - INFO - Starting the conversion process.
2024-09-06 09:27:26,296 - INFO - Starting the conversion process.
2024-09-06 09:27:26,296 - INFO - Starting the conversion process.
2024-09-06 09:27:26,298 - INFO - Starting the conversion process.
2024-09-06 09:27:26,303 - INFO - Starting the conversion process.
2024-09-06 09:27:26,305 - INFO - Starting the conversion process.
2024-09-06 09:27:26,307 - INFO - Starting the conversion process.
2024-09-06 09:27:26,308 - INFO - Starting the conversion process.
2024-09-06 09:27:26,309 - INFO - Starting the conversion process.
2024-09-06 09:27:26,311 - INFO - Starting the conversion process.
2024-09-06 09:27:26,313 - INFO - Starting the conversion process.
2024-09-06 09:27:26,314 - INFO - Starting the conversion process.
[WAR2024-0

-----------------generate Meta-Llama-3.1-8B-Instruct images -----------------


In [ ]:
model_answers['Meta-Llama-3.1-8B-Instruct']['0']

In [ ]:
model_answers['Meta-Llama-3.1-8B-Instruct']['0']['choices'][0]['turns'][0]['content']

In [ ]:
from datasets import load_dataset
def get_alpaca_eval_data(dataset="alpaca_eval_gpt4_baseline"):
    dataset = load_dataset(
        "tatsu-lab/alpaca_eval",
        dataset,
    )["eval"]
    return dataset
alpaca_eval = get_alpaca_eval_data()
def add_index(example, idx):  
    example['index'] = idx  
    return example  
dataset = alpaca_eval.map(add_index, with_indices=True) 

In [ ]:
alpaca_eval[:4]

In [ ]:
# 定义一个函数来添加索引  
def add_index(example, idx):  
    example['index'] = idx  
    return example  
dataset = alpaca_eval.map(add_index, with_indices=True) 
print(dataset[0])  # 打印第一行  

In [ ]:
dataset[0]

In [ ]:
from datasets import load_dataset  
from transformers import AutoTokenizer  
import hashlib  
import time  

from datasets import load_dataset
def get_alpaca_eval_data(dataset="alpaca_eval_gpt4_baseline"):
    dataset = load_dataset(
        "tatsu-lab/alpaca_eval",
        dataset,
    )["eval"]
    return dataset
alpaca_eval = get_alpaca_eval_data()
def add_index(example, idx):  
    example['index'] = idx  
    return example  
dataset = alpaca_eval.map(add_index, with_indices=True) 
# 加载数据集  
# dataset = load_dataset('alpaca_eval')  
  
# 定义一个函数来生成MD5哈希值  
def generate_md5(text):  
    return hashlib.md5(text.encode('utf-8')).hexdigest()  
  
# 使用预训练的分词器（例如，BERT的分词器）  
tokenizer_path = "/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/Meta-Llama-3.1-8B-Instruct" 
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)  
  
# 定义转换函数  
def transform_example(example, idx):  
    content = example['output']  
    tokenized = tokenizer(content)  
    token_len = len(tokenized['input_ids'])  
      
    transformed_example = {  
        "question_id": str(idx),  
        "answer_id": str(idx),  
        "model_id": example['generator'],  
        "choices": [  
            {  
                "index": 0,  
                "turns": [  
                    {  
                        "content": content,  
                        "token_len": token_len  
                    }  
                ]  
            }  
        ],  
        # "tstamp": time.time()  
    }  
    return transformed_example  
  
# 对数据集进行转换  
transformed_dataset = dataset.map(transform_example, with_indices=True, remove_columns=dataset.column_names)  
  
# 打印转换后的数据集的前几行  
print(transformed_dataset[0])  
transformed_dataset.to_json("gpt4_1106_preview.jsonl")


In [ ]:
transformed_dataset.to_json("temp_ref.jsonl")

In [ ]:
from datasets import load_dataset  
import hashlib  
  
# 加载数据集  
def get_alpaca_eval_data(dataset="alpaca_eval_gpt4_baseline"):  
    dataset = load_dataset("tatsu-lab/alpaca_eval", dataset)["eval"]  
    return dataset  
  
alpaca_eval = get_alpaca_eval_data()  
  
# 添加索引列  
def add_index(example, idx):  
    example['index'] = idx  
    return example  
  
questions = alpaca_eval.map(add_index, with_indices=True)  
  
# 定义转换函数  
def transform_example(example, idx):  
    transformed_example = {  
        "question_id": str(idx),  
        "category": "alpaca_eval",  
        "cluster": example['dataset'],  
        "turns": [  
            {  
                "content": example['instruction']  
            }  
        ]  
    }  
    return transformed_example  
  
# 对数据集进行转换  
transformed_questions = questions.map(transform_example, with_indices=True, remove_columns=dataset.column_names)  
  
# 打印转换后的数据集的前几行  
print(transformed_questions[0])  
transformed_questions.to_json("question.jsonl")


In [ ]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
import json
# template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"
template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"

model_name = "Meta-Llama-3.1-8B-Instruct"
# model_name = "Meta-Llama-3.1-8B"

# model_name = "tulu_lora_sft_default_template_8b"
# model_name = "tulu-2-dpo-7b"
# Create an LLM.
# llm = LLM(model=f"/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/{model_name}")

llm = LLM(model=f"/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints/{model_name}")
tokenizer = llm.get_tokenizer()
gen_kwargs_vllm = {
    "max_tokens": 2048,
    "top_p": 0.9,
    "top_k": 50,
    "temperature": 0.0,
    "repetition_penalty": 1.0,
}
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    # tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
    # tokenizer.chat_template
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")
    print("tokenizer is None, use setted template")
else:
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
    print("use original template")
# messages = tokenizer.apply_chat_template(messages, tokenize=False)



sampling_params = SamplingParams(**gen_kwargs_vllm)

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]
eval_set = eval_set.select(range(10))
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
eval_set = eval_set.map(convert_to_message)
# eval_set['messages']
outputs = llm.generate(eval_set['messages'], sampling_params)
print(outputs)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
# eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  
def rename_generator(sample):
    sample['generator'] = f"{model_name}"
    return sample
eval_set = eval_set.map(rename_generator)

In [ ]:
outputs[0].metrics

In [ ]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
import json
import argparse 

# parser = argparse.ArgumentParser(description='Set model name.')  
# parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
# parser.add_argument('--model-path', type=str, default="/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints", help='Dir of the model to use') 
# args = parser.parse_args()  
path_dir = "/mnt/lingjiejiang/textual_aesthetics/model_checkpoint/sft_merge_checkpoints" 
model_name = "Meta-Llama-3.1-8B-Instruct" 

# template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"
template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"

# model_name = "Meta-Llama-3.1-8B-Instruct"
# model_name = "Meta-Llama-3.1-8B"
# Create an LLM.
llm = LLM(model=f"{path_dir}/{model_name}")

print(f"model name: {model_name}")
print(f"model_path: {path_dir}/{model_name}")

gen_kwargs_vllm = {
    "max_tokens": 2048,
    "top_p": 0.9,
    "top_k": 50,
    "temperature": 0.0,
    "repetition_penalty": 1.0,
}
tokenizer = llm.get_tokenizer()
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
    # tokenizer.chat_template
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")
    print("tokenizer is None, use setted template")
else:
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
    print("use original template")
# messages = tokenizer.apply_chat_template(messages, tokenize=False)


sampling_params = SamplingParams(**gen_kwargs_vllm)

# 加载question.jsonl数据集  
questions = datasets.load_dataset('json', data_files='data/alpaca/question.jsonl')['train'] 
questions = questions.select(range(2)) 
print(questions[0])
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["turns"][0]['content']}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
questions = questions.map(convert_to_message)  

# 生成输出  
outputs = llm.generate(questions['messages'], sampling_params)  
outputs_text = [x.outputs[0].text for x in outputs]  
token_lens = [len(x.outputs[0].token_ids) for x in outputs]
# 移除现有的 'output' 列并添加新的 'output' 列  
questions = questions.remove_columns(["messages"])  
questions = questions.add_column("output", outputs_text)  
questions = questions.add_column("token_lens", token_lens) 

# 定义转换函数  
def transform_example(example):  
    content = example['output']  
    # tokenized = tokenizer(content)  
    token_len = example['token_lens']   
      
    transformed_example = {  
        "question_id": example['question_id'],  
        "answer_id": example['question_id'],  
        "model_id": model_name,  
        "choices": [  
            {  
                "index": 0,  
                "turns": [  
                    {  
                        "content": content,  
                        "token_len": token_len  
                    }  
                ]  
            }  
        ],  
        # "tstamp": time.time()  
    }  
    return transformed_example  
  
# 对数据集进行转换  
transformed_dataset = questions.map(transform_example, remove_columns=transform_example.column_names)  
  
# 保存为 JSONL 格式  
with open(f'./data/alpaca/model_answer/{model_name}.jsonl', 'w', encoding='utf-8') as f:  
    for example in transformed_dataset:  
        json.dump(example, f)  
        f.write('\n')  
  
print(f"Data saved to ./data/alpaca/model_answer/{model_name}.jsonl")  
 

In [ ]:
# 对数据集进行转换  
transformed_dataset = questions.map(transform_example, remove_columns=questions.column_names)  
  
# 保存为 JSONL 格式  
with open(f'./data/alpaca/model_answer/{model_name}.jsonl', 'w', encoding='utf-8') as f:  
    for example in transformed_dataset:  
        json.dump(example, f)  
        f.write('\n')  
  
print(f"Data saved to ./data/alpaca/model_answer/{model_name}.jsonl")  

In [ ]:

# 生成输出  
# outputs = llm.generate(questions['messages'], sampling_params)  
# outputs_text = [x.outputs[0].text for x in outputs]  
# token_lens = [len(x.outputs[0].token_ids) for x in outputs]
# # 移除现有的 'output' 列并添加新的 'output' 列  
# questions = questions.remove_columns(["messages"])  
# questions = questions.add_column("output", outputs_text)  
# questions = questions.add_column("token_lens", token_lens) 

# 定义转换函数  
def transform_example(example):  
    content = example['output']  
    # tokenized = tokenizer(content)  
    token_len = example['token_lens']   
      
    transformed_example = {  
        "question_id": example['question_id'],  
        "answer_id": example['question_id'],  
        "model_id": model_name,  
        "choices": [  
            {  
                "index": 0,  
                "turns": [  
                    {  
                        "content": content,  
                        "token_len": token_len  
                    }  
                ]  
            }  
        ],  
        # "tstamp": time.time()  
    }  
    return transformed_example  
  
# 对数据集进行转换  
transformed_dataset = questions.map(transform_example)  

In [ ]:
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["turns"][0]['content']}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
questions = questions.map(convert_to_message)  

# 生成输出  
outputs = llm.generate(questions['messages'], sampling_params) 

In [ ]:
from vllm import LLM, SamplingParams  
from tqdm import tqdm  
import datasets  
import json  
import argparse  
import hashlib  
import time  
  
# 解析命令行参数  
parser = argparse.ArgumentParser(description='Set model name.')  
parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use')  
args = parser.parse_args()  
path_dir = args.model_path  
model_name = args.model_name  
  
# 加载模型  
llm = LLM(model=f"{path_dir}/{model_name}")  
print(f"model name: {model_name}")  
print(f"model_path: {path_dir}/{model_name}")  
  
# 定义生成参数  
gen_kwargs_vllm = {  
    "max_tokens": 2048,  
    "top_p": 0.9,  
    "top_k": 50,  
    "temperature": 0.0,  
    "repetition_penalty": 1.0,  
}  
  
# 获取分词器  
tokenizer = llm.get_tokenizer()  
  
# 设置模板  
template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"  
  
if tokenizer.chat_template is None:  
    tokenizer.chat_template = template  
    tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)  
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]  
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")  
    print("tokenizer is None, use setted template")  
else:  
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]  
    print("use original template")  
  
sampling_params = SamplingParams(**gen_kwargs_vllm)  
  
# 加载数据集  
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]  
  
# 转换消息  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
eval_set = eval_set.map(convert_to_message)  
  
# 生成输出  
outputs = llm.generate(eval_set['messages'], sampling_params)  
outputs_text = [x.outputs[0].text for x in outputs]  
  
# 移除现有的 'output' 列并添加新的 'output' 列  
eval_set = eval_set.remove_columns(["output"])  
eval_set = eval_set.remove_columns(["messages"])  
eval_set = eval_set.add_column("output", outputs_text)  
  
# 重命名生成器  
def rename_generator(sample):  
    sample['generator'] = f"{model_name}"  
    return sample  
  
eval_set = eval_set.map(rename_generator)  
  
# 定义转换函数  
def transform_example(example, idx):  
    content = example['output']  
    tokenized = tokenizer(content)  
    token_len = len(tokenized['input_ids'])  
      
    transformed_example = {  
        "question_id": str(idx),  
        "answer_id": str(idx),  
        "model_id": example['generator'],  
        "choices": [  
            {  
                "index": 0,  
                "turns": [  
                    {  
                        "content": content,  
                        "token_len": token_len  
                    }  
                ]  
            }  
        ],  
        "tstamp": time.time()  
    }  
    return transformed_example  
  
# 对数据集进行转换  
transformed_dataset = eval_set.map(transform_example, with_indices=True)  
  
# 保存为 JSONL 格式  
with open(f'./data/{model_name}.jsonl', 'w', encoding='utf-8') as f:  
    for example in transformed_dataset:  
        json.dump(example, f)  
        f.write('\n')  
  
print(f"Data saved to ./data/{model_name}.jsonl")  


In [ ]:
import datasets  
questions = datasets.load_dataset('json', data_files='data/alpaca/question.jsonl')['train']
questions[:2]

In [ ]:
from vllm import LLM, SamplingParams  
from tqdm import tqdm  
import datasets  
import json  
import argparse  
import hashlib  
import time  
  
# 解析命令行参数  
parser = argparse.ArgumentParser(description='Set model name.')  
parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use')  
args = parser.parse_args()  
path_dir = args.model_path  
model_name = args.model_name  
  
# 加载模型  
llm = LLM(model=f"{path_dir}/{model_name}")  
print(f"model name: {model_name}")  
print(f"model_path: {path_dir}/{model_name}")  
  
# 定义生成参数  
gen_kwargs_vllm = {  
    "max_tokens": 2048,  
    "top_p": 0.9,  
    "top_k": 50,  
    "temperature": 0.0,  
    "repetition_penalty": 1.0,  
}  
  
# 获取分词器  
tokenizer = llm.get_tokenizer()  
  
# 设置模板  
template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"  
  
if tokenizer.chat_template is None:  
    tokenizer.chat_template = template  
    tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)  
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]  
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")  
    print("tokenizer is None, use setted template")  
else:  
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]  
    print("use original template")  
  
sampling_params = SamplingParams(**gen_kwargs_vllm)  
  
# 加载question.jsonl数据集  
questions = datasets.load_dataset('json', data_files='data/alpaca/question.jsonl')['train']  
  
# 转换消息  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["turns"][0]["content"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
questions = questions.map(convert_to_message)  
  
# 生成输出  
outputs = llm.generate(questions['messages'], sampling_params)  
outputs_text = [x.outputs[0].text for x in outputs]  
token_lens = [len(x.outputs[0].token_ids) for x in outputs]
# 移除现有的 'output' 列并添加新的 'output' 列  
questions = questions.remove_columns(["messages"])  
questions = questions.add_column("output", outputs_text)  
questions = questions.add_column("token_lens", token_lens) 
# 重命名生成器  
def rename_generator(sample):  
    sample['generator'] = f"{model_name}"  
    return sample  
  
questions = questions.map(rename_generator)  
  
# 定义转换函数  
def transform_example(example):  
    content = example['output']  
    # tokenized = tokenizer(content)  
    token_len = example['token_lens']   
      
    transformed_example = {  
        "question_id": example['question_id'],  
        "answer_id": example['question_id'],  
        "model_id": example['generator'],  
        "choices": [  
            {  
                "index": 0,  
                "turns": [  
                    {  
                        "content": content,  
                        "token_len": token_len  
                    }  
                ]  
            }  
        ],  
        "tstamp": time.time()  
    }  
    return transformed_example  
  
# 对数据集进行转换  
transformed_dataset = questions.map(transform_example)  
  
# 保存为 JSONL 格式  
with open(f'./data/{model_name}.jsonl', 'w', encoding='utf-8') as f:  
    for example in transformed_dataset:  
        json.dump(example, f)  
        f.write('\n')  
  
print(f"Data saved to ./data/{model_name}.jsonl")  


In [ ]:
from vllm import LLM, SamplingParams  
from tqdm import tqdm  
import datasets  
import json  
import argparse  
import hashlib  
import time  
  
# 解析命令行参数  
# parser = argparse.ArgumentParser(description='Set model name.')  
# parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
# parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use')  
# args = parser.parse_args()  
path_dir = "/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface"
model_name = args.model_name  
  
# 加载模型  
llm = LLM(model=f"{path_dir}/{model_name}")  
print(f"model name: {model_name}")  
print(f"model_path: {path_dir}/{model_name}")  
  
# 定义生成参数  
gen_kwargs_vllm = {  
    "max_tokens": 2048,  
    "top_p": 0.9,  
    "top_k": 50,  
    "temperature": 0.0,  
    "repetition_penalty": 1.0,  
}  
  
# 获取分词器  
tokenizer = llm.get_tokenizer()  
  
# 设置模板  
template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"  
  
if tokenizer.chat_template is None:  
    tokenizer.chat_template = template  
    tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)  
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]  
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")  
    print("tokenizer is None, use setted template")  
else:  
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]  
    print("use original template")  
  
sampling_params = SamplingParams(**gen_kwargs_vllm)  
  
# 加载question.jsonl数据集  
questions = datasets.load_dataset('json', data_files='data/alpaca/question.jsonl')['train']  
questions = questions.select(range(10))
# 转换消息  
def convert_to_message(example):  
    messages = [{"role": "user", "content": example["turns"][0]["content"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
  
questions = questions.map(convert_to_message)  
  
# 生成输出  
outputs = llm.generate(questions['messages'], sampling_params)  
outputs_text = [x.outputs[0].text for x in outputs]  